<a href="https://colab.research.google.com/github/yohanesnuwara/pyreservoir/blob/master/notebooks/pvt_correlation_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, ToggleButtons
import ipywidgets as widgets

In [2]:
!git clone https://github.com/yohanesnuwara/pyreservoir

Cloning into 'pyreservoir'...
remote: Enumerating objects: 207, done.
remote: Counting objects: 100% (207/207), done.
remote: Compressing objects: 100% (206/206), done.
remote: Total 441 (delta 107), reused 0 (delta 0), pack-reused 234
Receiving objects: 100% (441/441), 756.76 KiB | 2.17 MiB/s, done.
Resolving deltas: 100% (229/229), done.


In [3]:
import sys
sys.path.append('/content/pyreservoir/pvt')

from pvtcorrelation import *

## Gas

Input to gas PVT are:

* `pressure`: Pressure (psia)
* `temp`: Temperature (°F)
* `sg`: Specific gravity
* `x_h2s`: H2S mole fraction in gas
* `x_co2`: CO2 mole fraction in gas

In [4]:
help(gas_compressibility)

Help on function gas_compressibility in module pvtcorrelation:

gas_compressibility(T_pr, P_pr, rho_pr, z, P_pc)
    Calculate Gas Isothermal Compressibility
    For range: unspecified
    (Trube, 1957; Mattar, 1975)



In [5]:
"""
INPUT
"""

pressure = 2010 # psi
temp = 110 # Fahrenheit
sg = 0.5 # specific gravity
x_h2s = 0.07 # mole fraction of H2S in gas
x_co2 = 0.1 # mole fraction of CO2 in gas

"""
CALCULATOR
"""

# calculate pseudoproperties using Sutton (1985), Wichert and Aziz (1972)
P_pc, T_pc, P_pr, T_pr = gas_pseudoprops(temp, pressure, sg, x_h2s, x_co2)

# calculate z-factor using Dranchuk-Aboukassem (1975)
pseudo_rho, z_factor = gas_zfactor(T_pr, P_pr)

# calculate density
rhogas = gas_density(temp, pressure, sg, z_factor)

# calculate gas FVF
Bg = gas_fvf(z_factor, temp, pressure)

# calculate isothermal compressibility using Trube (1957) and Mattar (1975)
cgas = gas_compressibility(T_pr, P_pr, pseudo_rho, z_factor, P_pc)

# calculate viscosity using Lee et al (1966)
viscogas = gas_mu(temp, rhogas, sg)

print('=== Gas PVT Correlation Calculator === \n')
print('Your Input:')
print('Pressure                   : {} psia'.format(pressure))
print('Temperature                : {} °F'.format(temp))
print('Specific Gravity           : {}'.format(sg))
print('H2S Mole Fraction          : {}'.format(x_h2s))
print('CO2 Mole Fraction          : {} \n'.format(x_co2))

print('PVT Output:')
print('z-factor                   : {}'.format(z_factor))
print('Density                    : {} lb/ft3'.format(rhogas))
print('FVF                        : {} res ft3/scf'.format(Bg))
print('Isothermal compressibility : {} microsip'.format(cgas * 1E+6))
print('Viscosity                  : {} cp'.format(viscogas))


=== Gas PVT Correlation Calculator === 

Your Input:
Pressure                   : 2010 psia
Temperature                : 110 °F
Specific Gravity           : 0.5
H2S Mole Fraction          : 0.07
CO2 Mole Fraction          : 0.1 

PVT Output:
z-factor                   : nan
Density                    : nan lb/ft3
FVF                        : nan res ft3/scf
Isothermal compressibility : nan microsip
Viscosity                  : nan cp


In [6]:
help(gas_pseudoprops)

Help on function gas_pseudoprops in module pvtcorrelation:

gas_pseudoprops(temp, pressure, sg, x_h2s, x_co2)
    Calculate Gas Pseudo-critical and Pseudo-reduced Pressure and Temperature
    * Pseudo-critical properties
      For range: 0.57 < sg < 1.68
      (Sutton, 1985)
    * Pseudo-reduced properties
      For range: x_h2s (mol%) < 0.738; x_co2 (mol%) < 0.544; 154 < p (psia) < 7026; 40 < temp (°F) < 300 (error 0.97%)
      (Wichert and Aziz, 1972)



## Oil

Input to oil PVT are:

* `pressure`: Pressure (psia)
* `temp`: Temperature (°F)
* `sg`: Specific gravity
* `Rsb`: Solution gas-oil ratio at bubble-point pressure (scf/STB) (from PVT data)
* `api`: Oil gravity (API)

In [9]:
"""
INPUT
"""

pressure = 1900 # psi
temp = 220 # Fahrenheit
sg = 0.8 # specific gravity
Rsb = 500 # solution gas-oil ratio at bubble-point pressure (scf/STB) (from PVT data)
api = 30 # oil gravity (API)

"""
CALCULATOR
"""

# calculate bubble-point pressure using Vasquez and Beggs (1980)
pbubble = oil_pbubble(Rsb, sg, api, temp)

# calculate isothermal compressibility using Vazquez and Beggs (1980); McCain et al (1988)
coil = oil_compressibility(pressure, pbubble, temp, api, Rsb, sg)

# calculate FVF using Vazquez and Beggs (1980); Levitan and Murtha (1999)
Bo = oil_fvf(pbubble, api, Rsb, sg, temp, pressure)

# calculate gas-oil ratio using Vazquez and Beggs (1980)
Rs = gasoilratio(pressure, pbubble, sg, api, temp, Rsb)

# calculate gas-oil ratio using Vazquez and Beggs (1980); Beggs and Robinson (1975)
viscooil = oil_mu(pressure, pbubble, sg, api, temp, Rs)

print('=== Oil PVT Correlation Calculator === \n')
print('Your Input:')
print('Pressure                     : {} psia'.format(pressure))
print('Temperature                  : {} °F'.format(temp))
print('Specific Gravity             : {}'.format(sg))
print('Gas-oil ratio @ Bubble-point : {} scf/STB'.format(Rsb))
print('Oil gravity                  : {} API \n'.format(api))

print('PVT Output:')
print('Bubble-point Pressure        : {} psi'.format(pbubble))
print('Gas-oil ratio                : {} scf/STB'.format(Rs))
print('FVF                          : {} RB/STB'.format(Bo))
print('Isothermal compressibility   : {} microsip'.format(coil * 1E+6))
print('Viscosity                    : {} cp'.format(viscooil))


=== Oil PVT Correlation Calculator === 

Your Input:
Pressure                     : 1900 psia
Temperature                  : 220 °F
Specific Gravity             : 0.8
Gas-oil ratio @ Bubble-point : 500 scf/STB
Oil gravity                  : 30 API 

PVT Output:
Bubble-point Pressure        : 2650.3067919543523 psi
Gas-oil ratio                : 347.44320213156914 scf/STB
FVF                          : 1.2298060072933186 RB/STB
Isothermal compressibility   : 33.100146317783555 microsip
Viscosity                    : 0.7777699805921316 cp


## Water

Input to water PVT are:

* `pressure`: Pressure (psia)
* `temp`: Temperature (°F)
* `s`: salinity (wt%)

In [8]:
"""
INPUT
"""

pressure = 3000 # psi
temp = 190 # Fahrenheit
s = 5 # salinity, wt%

"""
CALCULATOR
"""

# calculate water FVF using McCain et al (1989)
Bw = water_fvf(temp, pressure)

# calculate vapor (bubble-point) pressure using the classic Antoine (1888)
pbubble = water_pbubble(temp)

# calculate isothermal water compressibility using Osif (1988) and McCain (1989)
cw = water_compressibility(temp, pressure, s, Bw)

# calculate water viscosity using McCain (1989)
mu_w = water_mu(temp, pressure, s)

print('=== Water PVT Correlation Calculator === \n')
print('Your Input:')
print('Pressure                     : {} psia'.format(pressure))
print('Temperature                  : {} °F'.format(temp))
print('Salinity                     : {} \n'.format(s / 100))

print('PVT Output:')
print('FVF                          : {} RB/STB'.format(Bw))
print('Bubble-Point Pressure        : {} psia'.format(pbubble))
print('Isothermal Compressibility   : {} microsip'.format(cw * 1E+6))
print('Viscosity                    : {} cp'.format(mu_w))


=== Water PVT Correlation Calculator === 

Your Input:
Pressure                     : 3000 psia
Temperature                  : 190 °F
Salinity                     : 0.05 

PVT Output:
FVF                          : 1.0305404582765743 RB/STB
Bubble-Point Pressure        : 9.327025034766681 psia
Isothermal Compressibility   : 3.10200919229241 microsip
Viscosity                    : 0.4326959802568671 cp
